In [48]:
import subprocess
import json
from tqdm import tqdm
from multiprocessing import Pool
import itertools
import pandas as pd

In [49]:
# resources = "AWS::EC2::CustomerGateway | AWS::EC2::EIP | AWS::EC2::Host | AWS::EC2::Instance | AWS::EC2::InternetGateway | AWS::EC2::NetworkAcl | AWS::EC2::NetworkInterface | AWS::EC2::RouteTable | AWS::EC2::SecurityGroup | AWS::EC2::Subnet | AWS::CloudTrail::Trail | AWS::EC2::Volume | AWS::EC2::VPC | AWS::EC2::VPNConnection | AWS::EC2::VPNGateway | AWS::EC2::RegisteredHAInstance | AWS::EC2::NatGateway | AWS::EC2::EgressOnlyInternetGateway | AWS::EC2::VPCEndpoint | AWS::EC2::VPCEndpointService | AWS::EC2::FlowLog | AWS::EC2::VPCPeeringConnection | AWS::Elasticsearch::Domain | AWS::IAM::Group | AWS::IAM::Policy | AWS::IAM::Role | AWS::IAM::User | AWS::ElasticLoadBalancingV2::LoadBalancer | AWS::ACM::Certificate | AWS::RDS::DBInstance | AWS::RDS::DBSubnetGroup | AWS::RDS::DBSecurityGroup | AWS::RDS::DBSnapshot | AWS::RDS::DBCluster | AWS::RDS::DBClusterSnapshot | AWS::RDS::EventSubscription | AWS::S3::Bucket | AWS::S3::AccountPublicAccessBlock | AWS::Redshift::Cluster | AWS::Redshift::ClusterSnapshot | AWS::Redshift::ClusterParameterGroup | AWS::Redshift::ClusterSecurityGroup | AWS::Redshift::ClusterSubnetGroup | AWS::Redshift::EventSubscription | AWS::SSM::ManagedInstanceInventory | AWS::CloudWatch::Alarm | AWS::CloudFormation::Stack | AWS::ElasticLoadBalancing::LoadBalancer | AWS::AutoScaling::AutoScalingGroup | AWS::AutoScaling::LaunchConfiguration | AWS::AutoScaling::ScalingPolicy | AWS::AutoScaling::ScheduledAction | AWS::DynamoDB::Table | AWS::CodeBuild::Project | AWS::WAF::RateBasedRule | AWS::WAF::Rule | AWS::WAF::RuleGroup | AWS::WAF::WebACL | AWS::WAFRegional::RateBasedRule | AWS::WAFRegional::Rule | AWS::WAFRegional::RuleGroup | AWS::WAFRegional::WebACL | AWS::CloudFront::Distribution | AWS::CloudFront::StreamingDistribution | AWS::Lambda::Function | AWS::NetworkFirewall::Firewall | AWS::NetworkFirewall::FirewallPolicy | AWS::NetworkFirewall::RuleGroup | AWS::ElasticBeanstalk::Application | AWS::ElasticBeanstalk::ApplicationVersion | AWS::ElasticBeanstalk::Environment | AWS::WAFv2::WebACL | AWS::WAFv2::RuleGroup | AWS::WAFv2::IPSet | AWS::WAFv2::RegexPatternSet | AWS::WAFv2::ManagedRuleSet | AWS::XRay::EncryptionConfig | AWS::SSM::AssociationCompliance | AWS::SSM::PatchCompliance | AWS::Shield::Protection | AWS::ShieldRegional::Protection | AWS::Config::ConformancePackCompliance | AWS::Config::ResourceCompliance | AWS::ApiGateway::Stage | AWS::ApiGateway::RestApi | AWS::ApiGatewayV2::Stage | AWS::ApiGatewayV2::Api | AWS::CodePipeline::Pipeline | AWS::ServiceCatalog::CloudFormationProvisionedProduct | AWS::ServiceCatalog::CloudFormationProduct | AWS::ServiceCatalog::Portfolio | AWS::SQS::Queue | AWS::KMS::Key | AWS::QLDB::Ledger | AWS::SecretsManager::Secret | AWS::SNS::Topic | AWS::SSM::FileData | AWS::Backup::BackupPlan | AWS::Backup::BackupSelection | AWS::Backup::BackupVault | AWS::Backup::RecoveryPoint | AWS::ECR::Repository | AWS::ECS::Cluster | AWS::ECS::Service | AWS::ECS::TaskDefinition | AWS::EFS::AccessPoint | AWS::EFS::FileSystem | AWS::EKS::Cluster | AWS::OpenSearch::Domain | AWS::EC2::TransitGateway | AWS::Kinesis::Stream | AWS::Kinesis::StreamConsumer | AWS::CodeDeploy::Application | AWS::CodeDeploy::DeploymentConfig | AWS::CodeDeploy::DeploymentGroup | AWS::EC2::LaunchTemplate | AWS::ECR::PublicRepository | AWS::GuardDuty::Detector | AWS::EMR::SecurityConfiguration | AWS::SageMaker::CodeRepository | AWS::Route53Resolver::ResolverEndpoint | AWS::Route53Resolver::ResolverRule | AWS::Route53Resolver::ResolverRuleAssociation | AWS::DMS::ReplicationSubnetGroup | AWS::DMS::EventSubscription | AWS::MSK::Cluster | AWS::StepFunctions::Activity | AWS::WorkSpaces::Workspace | AWS::WorkSpaces::ConnectionAlias | AWS::SageMaker::Model | AWS::ElasticLoadBalancingV2::Listener | AWS::StepFunctions::StateMachine | AWS::Batch::JobQueue | AWS::Batch::ComputeEnvironment | AWS::AccessAnalyzer::Analyzer | AWS::Athena::WorkGroup | AWS::Athena::DataCatalog | AWS::Detective::Graph | AWS::GlobalAccelerator::Accelerator | AWS::GlobalAccelerator::EndpointGroup | AWS::GlobalAccelerator::Listener | AWS::EC2::TransitGatewayAttachment | AWS::EC2::TransitGatewayRouteTable | AWS::DMS::Certificate | AWS::AppConfig::Application | AWS::AppSync::GraphQLApi | AWS::DataSync::LocationSMB | AWS::DataSync::LocationFSxLustre | AWS::DataSync::LocationS3 | AWS::DataSync::LocationEFS | AWS::DataSync::Task | AWS::DataSync::LocationNFS | AWS::EC2::NetworkInsightsAccessScopeAnalysis | AWS::EKS::FargateProfile | AWS::Route53::HostedZone"
resources = "AWS::EC2::CustomerGateway | AWS::EC2::EIP | AWS::EC2::Host | AWS::EC2::Instance | AWS::EC2::InternetGateway | AWS::EC2::NetworkAcl | AWS::EC2::NetworkInterface | AWS::EC2::RouteTable | AWS::EC2::SecurityGroup | AWS::EC2::Subnet | AWS::CloudTrail::Trail | AWS::EC2::Volume | AWS::EC2::VPC | AWS::EC2::VPNConnection | AWS::EC2::VPNGateway | AWS::EC2::RegisteredHAInstance | AWS::EC2::NatGateway | AWS::EC2::EgressOnlyInternetGateway | AWS::EC2::VPCEndpoint | AWS::EC2::VPCEndpointService | AWS::EC2::VPCPeeringConnection | AWS::IAM::Group | AWS::IAM::Policy | AWS::IAM::Role | AWS::IAM::User |  AWS::ElasticLoadBalancingV2::LoadBalancer | AWS::ACM::Certificate | AWS::RDS::DBInstance | AWS::RDS::DBSubnetGroup | AWS::RDS::DBSecurityGroup | AWS::RDS::DBSnapshot | AWS::RDS::DBCluster | AWS::RDS::DBClusterSnapshot | AWS::RDS::EventSubscription | AWS::S3::Bucket | AWS::S3::AccountPublicAccessBlock | AWS::ElasticLoadBalancing::LoadBalancer | AWS::AutoScaling::ScalingPolicy | AWS::AutoScaling::ScheduledAction | AWS::DynamoDB::Table | AWS::Lambda::Function | AWS::NetworkFirewall::Firewall | AWS::NetworkFirewall::FirewallPolicy | AWS::NetworkFirewall::RuleGroup | AWS::ECR::Repository | AWS::EKS::Cluster | AWS::Route53::HostedZone"
resource_lists = resources.split(" | ")

### 리소스 확인

In [50]:
exist_resources = []

def find_resource(resource) -> str:    
#     for item in tqdm(resource):
    proc = subprocess.Popen(
        ['aws', 'configservice', 'list-discovered-resources', '--resource-type', f'{resource}'],
    stdout = subprocess.PIPE
    )
    out, err = proc.communicate()
    
    if 'resourceType' in str(out):
        out = out.decode('utf-8')
        out = json.loads(out)
        resourceID = [out['resourceIdentifiers'][idx]['resourceId'] for idx in range(len(out['resourceIdentifiers']))]
        return [resource, resourceID]
    else:
        return 0


In [51]:
def multiprocessing(resource_lists) -> list:
    if __name__=='__main__':  
        pool = Pool(processes=12)
        resource = pool.map(find_resource, resource_lists)
        exist_resources.append(resource)
        print(resource)
        pool.close()
        pool.join()

In [52]:
multiprocessing(resource_lists)

[0, 0, 0, ['AWS::EC2::Instance', ['i-0303414e8cd5865c0', 'i-066189a9744d6ec3a', 'i-089772e9e1906ad02']], ['AWS::EC2::InternetGateway', ['igw-082d2f709fb46e0c1']], ['AWS::EC2::NetworkAcl', ['acl-09d6463116ca6874d']], ['AWS::EC2::NetworkInterface', ['eni-011b59f865f162856', 'eni-020dc0690f3e6a20e', 'eni-061cabcf0e433f314']], ['AWS::EC2::RouteTable', ['rtb-0bf37f5bb938ce786']], ['AWS::EC2::SecurityGroup', ['sg-0056aa002fd81c496', 'sg-014d3be3fb7e16872', 'sg-087fe725d638099cf', 'sg-0b8f402642ccea7fb']], ['AWS::EC2::Subnet', ['subnet-00411fd2c8840890e', 'subnet-08cffe1b0ae1f9b88', 'subnet-0f3206884d9698f40']], 0, ['AWS::EC2::Volume', ['vol-036f5e09497c71819', 'vol-0a6914dd0377cb8de', 'vol-0cb8e382bb271a77e']], ['AWS::EC2::VPC', ['vpc-03e53262faff38c42']], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['AWS::S3::Bucket', ['config-bucket-561123858318', 'mybucket-123421']], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [53]:
exist_resources = list(itertools.chain(*exist_resources))
exist_resources = [item for item in exist_resources if item != 0]
exist_resources

[['AWS::EC2::Instance',
  ['i-0303414e8cd5865c0', 'i-066189a9744d6ec3a', 'i-089772e9e1906ad02']],
 ['AWS::EC2::InternetGateway', ['igw-082d2f709fb46e0c1']],
 ['AWS::EC2::NetworkAcl', ['acl-09d6463116ca6874d']],
 ['AWS::EC2::NetworkInterface',
  ['eni-011b59f865f162856', 'eni-020dc0690f3e6a20e', 'eni-061cabcf0e433f314']],
 ['AWS::EC2::RouteTable', ['rtb-0bf37f5bb938ce786']],
 ['AWS::EC2::SecurityGroup',
  ['sg-0056aa002fd81c496',
   'sg-014d3be3fb7e16872',
   'sg-087fe725d638099cf',
   'sg-0b8f402642ccea7fb']],
 ['AWS::EC2::Subnet',
  ['subnet-00411fd2c8840890e',
   'subnet-08cffe1b0ae1f9b88',
   'subnet-0f3206884d9698f40']],
 ['AWS::EC2::Volume',
  ['vol-036f5e09497c71819', 'vol-0a6914dd0377cb8de', 'vol-0cb8e382bb271a77e']],
 ['AWS::EC2::VPC', ['vpc-03e53262faff38c42']],
 ['AWS::S3::Bucket', ['config-bucket-561123858318', 'mybucket-123421']]]

In [54]:
exist_resources = dict(zip(range(1, len(exist_resources) + 1), exist_resources))
json.dumps(exist_resources)

'{"1": ["AWS::EC2::Instance", ["i-0303414e8cd5865c0", "i-066189a9744d6ec3a", "i-089772e9e1906ad02"]], "2": ["AWS::EC2::InternetGateway", ["igw-082d2f709fb46e0c1"]], "3": ["AWS::EC2::NetworkAcl", ["acl-09d6463116ca6874d"]], "4": ["AWS::EC2::NetworkInterface", ["eni-011b59f865f162856", "eni-020dc0690f3e6a20e", "eni-061cabcf0e433f314"]], "5": ["AWS::EC2::RouteTable", ["rtb-0bf37f5bb938ce786"]], "6": ["AWS::EC2::SecurityGroup", ["sg-0056aa002fd81c496", "sg-014d3be3fb7e16872", "sg-087fe725d638099cf", "sg-0b8f402642ccea7fb"]], "7": ["AWS::EC2::Subnet", ["subnet-00411fd2c8840890e", "subnet-08cffe1b0ae1f9b88", "subnet-0f3206884d9698f40"]], "8": ["AWS::EC2::Volume", ["vol-036f5e09497c71819", "vol-0a6914dd0377cb8de", "vol-0cb8e382bb271a77e"]], "9": ["AWS::EC2::VPC", ["vpc-03e53262faff38c42"]], "10": ["AWS::S3::Bucket", ["config-bucket-561123858318", "mybucket-123421"]]}'

In [107]:
df_fine_resource = pd.DataFrame.from_dict(exist_resources, orient='index', columns=['Resource_Type','ResourceID'])
df_fine_resource['date']=pd.datetime.now().date()
df_fine_resource.explode('ResourceID')

/tmp/ipykernel_6286/370246546.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df_fine_resource['date']=pd.datetime.now().date()


,Resource_Type,ResourceID,date
1,AWS::EC2::Instance,i-0303414e8cd5865c0,2023-03-09
1,AWS::EC2::Instance,i-066189a9744d6ec3a,2023-03-09
1,AWS::EC2::Instance,i-089772e9e1906ad02,2023-03-09
2,AWS::EC2::InternetGateway,igw-082d2f709fb46e0c1,2023-03-09
3,AWS::EC2::NetworkAcl,acl-09d6463116ca6874d,2023-03-09
4,AWS::EC2::NetworkInterface,eni-011b59f865f162856,2023-03-09
4,AWS::EC2::NetworkInterface,eni-020dc0690f3e6a20e,2023-03-09
4,AWS::EC2::NetworkInterface,eni-061cabcf0e433f314,2023-03-09
5,AWS::EC2::RouteTable,rtb-0bf37f5bb938ce786,2023-03-09
6,AWS::EC2::SecurityGroup,sg-0056aa002fd81c496,2023-03-09


### 세부 정보

In [11]:
for item in range(len(exist_resources[0][0]))

proc = subprocess.Popen(
    ['aws', 'configservice', 'get-resource-config-history', '--resource-type', exist_resources[0][0], '--resource-id', exist_resources[0][1][1]],
    stdout = subprocess.PIPE
    )
out, err = proc.communicate()

In [12]:
result = out.decode('utf-8')
result = json.loads(result)
result

{'configurationItems': [{'version': '1.3',
   'accountId': '561123858318',
   'configurationItemCaptureTime': '2023-03-09T17:29:57.787000+09:00',
   'configurationItemStatus': 'OK',
   'configurationStateId': '1678350597787',
   'configurationItemMD5Hash': '',
   'arn': 'arn:aws:ec2:us-east-2:561123858318:instance/i-066189a9744d6ec3a',
   'resourceType': 'AWS::EC2::Instance',
   'resourceId': 'i-066189a9744d6ec3a',
   'awsRegion': 'us-east-2',
   'availabilityZone': 'us-east-2a',
   'resourceCreationTime': '2023-03-09T17:17:55+09:00',
   'tags': {'Name': 'test1'},
   'relatedEvents': [],
   'relationships': [{'resourceType': 'AWS::EC2::Volume',
     'resourceId': 'vol-036f5e09497c71819',
     'relationshipName': 'Is attached to Volume'},
    {'resourceType': 'AWS::EC2::NetworkInterface',
     'resourceId': 'eni-011b59f865f162856',
     'relationshipName': 'Contains NetworkInterface'},
    {'resourceType': 'AWS::EC2::Subnet',
     'resourceId': 'subnet-08cffe1b0ae1f9b88',
     'relation

In [13]:
config = result['configurationItems'][0]['configuration']
config = json.loads(config)
config

{'amiLaunchIndex': 0,
 'imageId': 'ami-0f3c9c466bb525749',
 'instanceId': 'i-066189a9744d6ec3a',
 'instanceType': 't2.micro',
 'kernelId': None,
 'keyName': None,
 'launchTime': '2023-03-09T08:17:55.000Z',
 'monitoring': {'state': 'disabled'},
 'placement': {'availabilityZone': 'us-east-2a',
  'affinity': None,
  'groupName': '',
  'partitionNumber': None,
  'hostId': None,
  'tenancy': 'default',
  'spreadDomain': None,
  'hostResourceGroupArn': None,
  'groupId': None},
 'platform': None,
 'privateDnsName': 'ip-172-31-9-51.us-east-2.compute.internal',
 'privateIpAddress': '172.31.9.51',
 'productCodes': [],
 'publicDnsName': '',
 'publicIpAddress': None,
 'ramdiskId': None,
 'state': {'code': 80, 'name': 'stopped'},
 'stateTransitionReason': 'User initiated (2023-03-09 08:28:01 GMT)',
 'subnetId': 'subnet-08cffe1b0ae1f9b88',
 'vpcId': 'vpc-03e53262faff38c42',
 'architecture': 'x86_64',
 'blockDeviceMappings': [{'deviceName': '/dev/xvda',
   'ebs': {'attachTime': '2023-03-09T08:17:56.

### DataFrame

In [206]:
df_config = pd.DataFrame.from_dict(config, orient='index')
df_config.T

,amiLaunchIndex,imageId,instanceId,instanceType,kernelId,keyName,launchTime,monitoring,placement,platform,...,metadataOptions,enclaveOptions,bootMode,platformDetails,usageOperation,usageOperationUpdateTime,privateDnsNameOptions,ipv6Address,tpmSupport,maintenanceOptions
0,0,ami-0f3c9c466bb525749,i-0303414e8cd5865c0,t2.micro,None,None,2023-03-06T13:56:39.000Z,{'state': 'disabled'},"{'availabilityZone': 'us-east-2a', 'affinity':...",None,...,"{'state': 'applied', 'httpTokens': 'optional',...",{'enabled': False},None,None,None,None,None,None,None,None


In [108]:
import boto3
import json

REGIONS = [
    "ap-northeast-2",
    "us-east-2"
]

def get_ec2_id(region_name):
    ec2 = boto3.client('ec2', region_name=region_name)
    ec2s = ec2.describe_instances()
    ec2s_configs = ec2s['Reservations']

    return ec2s_configs

In [109]:
test = get_ec2_id("us-east-2")


[{'Groups': [],
  'Instances': [{'AmiLaunchIndex': 0,
    'ImageId': 'ami-0f3c9c466bb525749',
    'InstanceId': 'i-089772e9e1906ad02',
    'InstanceType': 't2.micro',
    'LaunchTime': datetime.datetime(2023, 3, 9, 8, 29, 33, tzinfo=tzutc()),
    'Monitoring': {'State': 'disabled'},
    'Placement': {'AvailabilityZone': 'us-east-2a',
     'GroupName': '',
     'Tenancy': 'default'},
    'PrivateDnsName': 'ip-172-31-12-122.us-east-2.compute.internal',
    'PrivateIpAddress': '172.31.12.122',
    'ProductCodes': [],
    'PublicDnsName': '',
    'State': {'Code': 80, 'Name': 'stopped'},
    'StateTransitionReason': 'User initiated (2023-03-09 08:29:54 GMT)',
    'SubnetId': 'subnet-08cffe1b0ae1f9b88',
    'VpcId': 'vpc-03e53262faff38c42',
    'Architecture': 'x86_64',
    'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
      'Ebs': {'AttachTime': datetime.datetime(2023, 3, 9, 8, 29, 34, tzinfo=tzutc()),
       'DeleteOnTermination': True,
       'Status': 'attached',
       'VolumeId'

In [110]:
type(test)

list

In [114]:
df_config = pd.DataFrame(test)
df_config = df_config.explode('Instances')

In [115]:
df_config['Instances']

0    {'AmiLaunchIndex': 0, 'ImageId': 'ami-0f3c9c46...
1    {'AmiLaunchIndex': 0, 'ImageId': 'ami-0f3c9c46...
2    {'AmiLaunchIndex': 0, 'ImageId': 'ami-0f3c9c46...
Name: Instances, dtype: object

In [90]:
df_config = pd.DataFrame.from_list(test, orient='index').T

AttributeError: type object 'DataFrame' has no attribute 'from_list'

In [83]:
df_config

,Reservations,ResponseMetadata
0,"{'Groups': [], 'Instances': [{'AmiLaunchIndex'...",RequestId
1,"{'Groups': [], 'Instances': [{'AmiLaunchIndex'...",HTTPStatusCode
2,"{'Groups': [], 'Instances': [{'AmiLaunchIndex'...",HTTPHeaders
3,None,RetryAttempts


In [82]:
df_config.explode('Reservations')

,Reservations,ResponseMetadata
0,Groups,RequestId
0,Instances,RequestId
0,OwnerId,RequestId
0,ReservationId,RequestId
1,Groups,HTTPStatusCode
1,Instances,HTTPStatusCode
1,OwnerId,HTTPStatusCode
1,ReservationId,HTTPStatusCode
2,Groups,HTTPHeaders
2,Instances,HTTPHeaders
